<a href="https://colab.research.google.com/github/jkranyak/birdscompetitionkaggle/blob/main/notebooka3489e63aa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'birdclef-2024:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F70203%2F8068726%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240411%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240411T234502Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D46e0e22533c4fcd389c01d0e4a59680f0195c48981d89b38fbcc56169788c4e2ec0cd598173d4b060798187fc098d9ae8be44f928e19332fd7a04dd9291b7cd5812e6ccb7a55cde8fb478dd218fbb7838d1668800383c690cadf71fdb848aef8fc877fe66aa901c88f53b367b5159910d89ce41d7d6e5efaa398ab4edb3a3698eb791cf66685b93ce0b06ccbe72381b605d5a1cd8666e4a5b0c3deed470d2e40bcebf7459e89a08001c6c206049a51e26d8448f234521076fa17a96a87092b56e9f57708917dce91638fcc740d304c9522406f9230270c574117522f32a1760132f49b58bdb9ccae232fd02fe940370249bd14d548dc193e0be81f80f0badebf'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[=============================                     ] 13794672640 bytes downloaded

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install librosa soundfile tensorflow scikit-learn numpy pandas matplotlib seaborn tqdm
!pip install audiomentations
!pip install tqdm


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Flatten, LSTM, TimeDistributed, Dense, Dropout, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from keras.utils import Sequence
from keras.preprocessing.image import load_img, img_to_array

import os
import librosa
import librosa.display
import soundfile as sf
import torch
import torch.nn as nn
from tqdm import tqdm

import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
import multiprocessing as mp

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

import joblib
from multiprocessing import Pool
from scipy import signal
from scipy.io import wavfile
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift


In [ ]:
k# Define output directories
output_directories = ['processed_data', 'models', 'submissions', 'visualizations']
base_working_dir = '/kaggle/working/'

# Creating subdirectories
for directory in output_directories:
    path = os.path.join(base_working_dir, directory)
    os.makedirs(path, exist_ok=True)
    print(f"Created directory: {path}")

# Define base directories for input data
base_input_dir = "/kaggle/input/birdclef-2024"

# Paths for specific data types
train_audio_base_path = os.path.join(base_input_dir, "train_audio")
unlabeled_soundscapes_path = os.path.join(base_input_dir, "unlabeled_soundscapes")
train_metadata_csv_path = os.path.join(base_input_dir, "train_metadata.csv")
taxonomy_csv_path = os.path.join(base_input_dir, "eBird_Taxonomy_v2021.csv")
sample_submission_csv_path = os.path.join(base_input_dir, "sample_submission.csv")
test_soundscapes_path = os.path.join(base_input_dir, "test_soundscapes")

# Function to list all audio files considering subdirectories
def list_all_audio_files(base_path):
    all_files = []
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith('.ogg'):
                all_files.append(os.path.join(root, file))
    return all_files

# Example usage
all_train_audio_files = list_all_audio_files(train_audio_base_path)
print(f"Total audio files found: {len(all_train_audio_files)}")


### Audio Processing and Spectrogram Generation

In this section, we focus on converting raw audio data into a more visually interpretable form called a spectrogram, which is crucial for our Convolutional Recurrent Neural Network (CRNN) model. Spectrograms provide a 2D representation of the sound where the x-axis represents time, the y-axis represents frequency, and the intensity of colors shows the amplitude of a particular frequency at a particular time.

#### Step-by-step Process:

1. **Audio Loading**: Each audio file is loaded into the memory, which captures the sound's waveform as a time-series data.
2. **Spectrogram Conversion**: The raw audio is then converted into a spectrogram using the Mel scale, which is more aligned with human auditory perception.
3. **Visualization and Saving**: The generated spectrogram is visualized and saved as a PNG file. This image serves as the input to our neural network.

The following visualization is an example of a spectrogram generated from our dataset. This particular audio file represents the call of a bird species recorded in the Western Ghats. By analyzing such images, our model learns to identify patterns specific to different bird calls.





In [ ]:
# Example of generating a spectrogram
import librosa.display
import matplotlib.pyplot as plt

# Load an example audio file
y, sr = librosa.load(librosa.ex('trumpet'))

# Create a Mel-spectrogram
S = librosa.feature.melspectrogram(y=y, sr=sr)
S_DB = librosa.power_to_db(S, ref=np.max)

# Plot the Mel-spectrogram
plt.figure(figsize=(10, 4))
librosa.display.specshow(S_DB, sr=sr, x_axis='time', y_axis='mel')
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-frequency spectrogram')
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
from tqdm import tqdm
# Load metadata
metadata_path = '/kaggle/input/birdclef-2024/train_metadata.csv'
metadata = pd.read_csv(metadata_path)
filtered_metadata = metadata[metadata['latitude'].between(8, 21) & metadata['longitude'].between(72, 78)]

# Define the base directory for train audio
base_audio_dir = '/kaggle/input/birdclef-2024/train_audio'
output_base_dir = '/kaggle/working/processed_data'

# Check and construct full audio paths, verify existence
filtered_metadata.loc[:, 'audio_path'] = filtered_metadata.apply(lambda row: create_audio_path(row, base_audio_dir), axis=1)
filtered_metadata.loc[:, 'exists'] = filtered_metadata['audio_path'].apply(os.path.exists)

# Filter out non-existing files
valid_files_metadata = filtered_metadata[filtered_metadata['exists']]

def process_and_save_spectrogram(row):
    """Process each audio file, generate spectrogram, and save in designated folder."""
    audio_path = row['audio_path']
    primary_label = row['primary_label'] if pd.notna(row['primary_label']) else 'unknown'
    output_dir = os.path.join(output_base_dir, primary_label)
    os.makedirs(output_dir, exist_ok=True)

    try:
        y, sr = librosa.load(audio_path, sr=None)
        S = librosa.feature.melspectrogram(y=y, sr=sr)
        S_DB = librosa.power_to_db(S, ref=np.max)
        plt.figure(figsize=(10, 4))
        librosa.display.specshow(S_DB, sr=sr, x_axis='time', y_axis='mel')
        plt.colorbar(format='%+2.0f dB')
        plt.title(f'Mel-frequency spectrogram of {primary_label}')
        plt.tight_layout()

        save_path = os.path.join(output_dir, f"{os.path.splitext(os.path.basename(audio_path))[0]}_spectrogram.png")
        plt.savefig(save_path)
        plt.close()
        return f"Saved: {save_path}"
    except Exception as e:
        return f"Failed to process {audio_path}: {str(e)}"

# Process and save spectrograms
results = [process_and_save_spectrogram(row) for _, row in tqdm(valid_files_metadata.iterrows(), total=valid_files_metadata.shape[0])]

# Optionally print the results
for result in results[:10]:  # Just print the first 10 results
    print(result)

### Model Design: Convolutional Recurrent Neural Network (CRNN)

The CRNN model combines the spatial feature extraction capabilities of Convolutional Neural Networks (CNNs) with the sequence modeling prowess of Recurrent Neural Networks (RNNs), making it ideal for audio classification tasks like bird call identification.

#### Architecture Overview:

- **Convolutional Layers**: These layers extract a hierarchy of high-level features from the spectrogram images.
- **Recurrent Layer**: An LSTM layer processes the sequence of features extracted by the CNN to capture temporal dependencies.
- **Output Layer**: Outputs the probability distribution across all bird species, predicting the likelihood of each species being present in the audio clip.

#### Why CRNN?

- **Spatial Feature Extraction**: The CNN layers effectively identify spatial features in spectrograms, such as shapes and textures associated with various bird sounds.
- **Temporal Sequence Modeling**: The LSTM layer captures the temporal dynamics of bird calls, crucial for distinguishing between similar-sounding bird species.
- **Efficiency**: CRNNs can handle large input sizes and are effective in environments with limited labeled data, which is typical in bird sound classification.

This hybrid model structure leverages the strengths of both CNNs and RNNs, providing a robust framework for our bird species classification task.

The next steps will involve training this model on our processed dataset, evaluating its performance, and refining the approach based on the observed results.


In [ ]:
def create_crnn_model(input_shape, num_classes):
    model = Sequential([
        TimeDistributed(Conv2D(32, kernel_size=(3, 3), activation='relu'), input_shape=input_shape),
        TimeDistributed(BatchNormalization()),
        TimeDistributed(MaxPooling2D(pool_size=(2, 2))),
        TimeDistributed(Conv2D(64, kernel_size=(3, 3), activation='relu')),
        TimeDistributed(BatchNormalization()),
        TimeDistributed(MaxPooling2D(pool_size=(2, 2))),
        TimeDistributed(Flatten()),
        LSTM(64, return_sequences=False),
        Dense(64, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Define model parameters
input_shape = (None, 128, 128, 1)  # Example input shape (time_steps, height, width, channels)
num_classes = 182  # Number of bird species

# Create the CRNN model
model = create_crnn_model(input_shape, num_classes)
model.summary()


In [ ]:
import os
import numpy as np
from PIL import Image

# Define the directory where spectrogram images are saved
processed_data_dir = '/kaggle/working/processed_data'

# Function to load spectrogram images from a directory
def load_spectrogram_images(directory):
    spectrogram_images = []
    labels = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('_spectrogram.png'):
                # Load the image
                image_path = os.path.join(root, file)
                image = Image.open(image_path)
                # Convert to numpy array and normalize
                spectrogram = np.array(image) / 255.0  # Normalize to [0, 1]
                # Append to the list of spectrogram images
                spectrogram_images.append(spectrogram)
                # Extract the label from the directory name
                label = os.path.basename(root)
                labels.append(label)
    return np.array(spectrogram_images), np.array(labels)

# Load spectrogram images and corresponding labels
features, labels = load_spectrogram_images(processed_data_dir)

# Print the shape of features and labels
print("Shape of features:", features.shape)
print("Number of labels:", len(labels))


### Explanation of the Code

#### Model Setup and Compilation
- **Importing Libraries:**
  - The code begins by importing necessary modules from TensorFlow and Matplotlib.
- **Hyperparameters:**
  - Learning rate, epochs, and batch size are defined.
- **Model Compilation:**
  - The model is compiled using the Adam optimizer with a specified learning rate and categorical cross-entropy loss function. Accuracy is chosen as the evaluation metric.

#### Callbacks Setup
- **Model Checkpoint:**
  - This callback saves the model weights to 'best_model.h5' file when validation accuracy improves.
- **Early Stopping:**
  - This callback monitors validation loss and stops training if there's no improvement for a specified number of epochs (patience).

#### Model Training
- **Training the Model:**
  - The `fit()` function is called to train the model on the training dataset (`train_dataset`) for the specified number of epochs. Validation data (`val_dataset`) is provided for monitoring the model's performance during training. Callbacks are passed to this function for checkpointing and early stopping.

#### Visualization
- **Training History Plotting:**
  - Matplotlib is used to visualize the training and validation accuracy over epochs. This helps in assessing the model's performance and identifying overfitting or underfitting issues.


In [ ]:
# # Callbacks Setup

# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# # Model Checkpoint callback
# checkpoint_filepath = '/kaggle/working/models/best_model.h5'
# model_checkpoint_callback = ModelCheckpoint(
#     filepath=checkpoint_filepath,
#     save_weights_only=True,
#     monitor='val_accuracy',
#     mode='max',
#     save_best_only=True)

# # Early Stopping callback
# early_stopping_callback = EarlyStopping(
#     monitor='val_loss',
#     patience=5,
#     restore_best_weights=True
# )

# # Model Training

# history = model.fit(
#     train_dataset,
#     epochs=50,
#     validation_data=val_dataset,
#     callbacks=[model_checkpoint_callback, early_stopping_callback]
# )

# # Visualization

# import matplotlib.pyplot as plt

# # Plot training & validation accuracy values
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='upper left')
# plt.show()

# # Plot training & validation loss values
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='upper left')
# plt.show()
